In [1]:
import numpy as np
import matplotlib.pyplot as plt

import some_routines as sr

# 8. Evolving the simulation

#### We use the particle mesh (PM) algorithm. In general they solve the Poisson equation given by
\begin{equation}
\nabla^2 \Phi = 4\pi G\rho_{tot} - \Lambda
\end{equation}
#### and the equations of motion of the particles are given by 
\begin{equation}
\frac{d\vec{r}}{dt} = \vec{u}, \frac{d\vec{u}}{dt} = -\nabla \Phi
\end{equation}
#### All equations are given in proper coordinates including all derivatives. It is however convenient to use comoving variables and dimensionless units. Lets define the dimensionless units with a tilde as given in the assignment. 

#### We will choose $r_0$ to be equal to the size of a PM grid cell.
\begin{equation}
$r_0 = \frac{L_{box}}{N_g}$
\end{equation}
#### This also defines the rest of the units.
#### Furthermore it is convenient to take the expansion factor as the time variable, in this case the equations of motions are as given in assignment Eq. 23 and 24, see also Eq. 25. The dimensionless equations are given in Eq. 26. Thus if we use leapfrog the equations are given by.. 
\begin{equation}
\tilde{\vec{p}}_{n+1/2} = \tilde{\vec{p}}_{n+1/2} + f(a) \tilde{g} \Delta a
\end{equation}
\begin{equation}
\tilde{\vec{x}}_{n+1} = \tilde{\vec{x}} + \frac{1}{a_{n+1/2}} f(a_{n+1/2})\tilde{\vec{p}}_{n+1/2} \Delta a
\end{equation}
In which $g = -\tilde{\nabla}\tilde{\phi}.$


### Now,

#### Write your own PM algorithm code using the information above. Use a time stepping size of $\Delta a = 1e-4$ and start at $z=50$. Output at least 300 hdf5 snapshots to make a movie of the simulation. The code should be in the top folder. Code may take up to 30 mins. Allowed to use as initial conditions 64^3 randomly places particles with zero velocity. Use $\Omega_\Lambda = 0.7$ and $Omega_m = 0.3$. Movie of at least 10 seconds with 30 FPS. Make it such that the angle rotates from 0 to 90 degrees in 10 seconds.

In [8]:
class CIC(object):
    def __init__(self, nperdim, positions):
        """
        nperdim   -- int: amount of gridpoints per dimension
        positions -- array: 3D positions of the particles (3,?)
        """
        self.nperdim = nperdim
        # Particle positions [xvalues,yvalues,zvalues]
        self.positions = positions
        # cell size
        self.deltax = 1
        # Masses of the gridpoints
        self.massgrid = self.assign_mass()
        
    def assign_mass(self):
        """
        Use the CIC method to assign mass to all the cells.
        Loop over all particles.
        """
        massgrid = np.zeros((self.nperdim,self.nperdim,self.nperdim))
        # Index of the cell containing the particle is floored int
        indices = np.array(self.positions, dtype='int')
        # Distances to nearest grid point in all directions
        # The center of cell 'ijk' is ijk+0.5. 
        # (e.g., (0.5,0.5,0.5_ for cell 0)
        dr = self.positions-(indices+0.5)
        # Check if neighbour is the next or previous cell
        # If the distance is positive, we need to add mass to
        # the next cell. If the distance is negative, to the previous
        # Thus, next index is the sign of the distance
        next_idx = np.array(np.sign(dr),dtype='int')    
        # Then make sure we know distances are always positive
        dr = np.abs(dr)
        tr = 1-dr
        
        # Assign each particle mass to 8 gridpoints
        for i in range(indices.shape[1]):
            indx = indices[:,i]
            # Current cell
            massgrid[tuple(indx)] += np.prod(tr[:,i])
            # Neighbours. Note periodic boundary conditions
            nb = next_idx[:,0]
            
            idx = tuple((indx+np.array([nb[0],0,0]))%self.nperdim)
            massgrid[idx] += dr[0,i]*tr[1,i]*tr[2,i]
            idx = tuple((indx+np.array([0,nb[1],0]))%self.nperdim)
            massgrid[idx] += tr[0,i]*dr[1,i]*tr[2,i]
            idx = tuple((indx+np.array([nb[0],nb[1],0]))%self.nperdim)
            massgrid[idx] += dr[0,i]*dr[1,i]*tr[2,i]
            idx = tuple((indx+np.array([0,0,nb[2]]))%self.nperdim)
            massgrid[idx] += tr[0,i]*tr[1,i]*dr[2,i]
            idx = tuple((indx+np.array([nb[0],0,nb[2]]))%self.nperdim)
            massgrid[idx] += dr[0,i]*tr[1,i]*dr[2,i]
            idx = tuple((indx+np.array([0,nb[1],nb[2]]))%self.nperdim)
            massgrid[idx] += tr[0,i]*dr[1,i]*dr[2,i]
            idx = tuple((indx+np.array([nb[0],nb[1],nb[2]]))%self.nperdim)
            massgrid[idx] += dr[0,i]*dr[1,i]*dr[2,i]
        return massgrid
    
    def compute_force(self, pindices, potential):
        """
        Return gradient of the potential 
        (= -1*Force for unit mass particles.)
        The gradient of the potential is calculated with inverse CIC
        interpolation.
        
        pindices  -- list of integers, the indices of the particles
                     to compute the gradient for.
                     --> must be between 0 and len(self.positions)
                     
        potential -- 3D array, the potential of every cell
        
        Returns
        all_gradients -- array of shape (len(pindices),3)
                         containing for every particle the 3D gradient
        """
        # Calculate gradient in x direction of every cell
        # with central difference method
        Fx = (np.roll(potential,-1,axis=0) - np.roll(potential,1,axis=0))/(2*self.deltax)
        # Repeat for y and z direction, shape (Nperdim,Nperdim,Nperdim)
        Fy = (np.roll(potential,-1,axis=1) - np.roll(potential,1,axis=1))/(2*self.deltax)
        Fz = (np.roll(potential,-1,axis=2) - np.roll(potential,1,axis=2))/(2*self.deltax)
        
        gradpot = np.zeros((self.nperdim,self.nperdim,self.nperdim,3))
        gradpot[:,:,:,0] = Fx
        gradpot[:,:,:,1] = Fy
        gradpot[:,:,:,2] = Fz
        
        # Indices of the cells containing the particles is floored int
        indices = np.array(self.positions[:,pindices], dtype='int')

        # Equivalent to mass assignment scheme
        dr = self.positions[:,pindices]-indices
        next_idx = np.array(np.sign(dr),dtype='int')    
        dr = np.abs(dr)
        tr = 1-dr
        
        # Interpolate each of the 8 gridpoints contribution to the
        # single value for the gradient of the potential
        all_gradients = np.zeros((len(pindices),3)) # each particle has 3D gradient 
        for i in range(len(pindices)):
            # parent cell
            indx = indices[:,i]
            grad = gradpot[tuple(indx)]*np.prod(tr[:,i])
            # Neighbours. Note periodic boundary conditions
            nb = next_idx[:,0]
            
            idx = tuple((indx+np.array([nb[0],0,0]))%self.nperdim)
            grad += gradpot[idx]*dr[0,i]*tr[1,i]*tr[2,i]
            
            idx = tuple((indx+np.array([0,nb[1],0]))%self.nperdim)
            grad += gradpot[idx]*tr[0,i]*dr[1,i]*tr[2,i]
            
            idx = tuple((indx+np.array([nb[0],nb[1],0]))%self.nperdim)
            grad += gradpot[idx]*dr[0,i]*dr[1,i]*tr[2,i]
            
            idx = tuple((indx+np.array([0,0,nb[2]]))%self.nperdim)
            grad += gradpot[idx]*tr[0,i]*tr[1,i]*dr[2,i]
            
            idx = tuple((indx+np.array([nb[0],0,nb[2]]))%self.nperdim)
            grad += gradpot[idx]*dr[0,i]*tr[1,i]*dr[2,i]
            
            idx = tuple((indx+np.array([0,nb[1],nb[2]]))%self.nperdim)
            grad += gradpot[idx]*tr[0,i]*dr[1,i]*dr[2,i]
            
            idx = tuple((indx+np.array([nb[0],nb[1],nb[2]]))%self.nperdim)
            grad += gradpot[idx]*dr[0,i]*dr[1,i]*dr[2,i]
            
            all_gradients[i,:] = grad
            
        return all_gradients
    
def kvecsquared(N, ndim):
    """
    Generate NxN(xN) matrix of k**2 values to multiply with a grid
    """
    dk = 2*np.pi/N
    ks = np.zeros(N) # aranged vector of kx modes
    # Loop over all kx modes
    for i in range(0,N): 
        if i <= N//2:
            ks[i] = dk*i
        else:
            ks[i] = (-N+i)*dk
            
    # My implementation has a different definition
    # for the x axis than numpy, thus swap y and x from np.meshgrid
    if ndim == 2:
        # every particle has a 2D position
        kvector = np.zeros((N,N,ndim))
        # simply replaces more of the same for loops
        ky, kx = np.meshgrid(ks,ks) # construct a grid
        ksquared = kx**2 + ky**2
    elif ndim == 3:
        # every particle has a 3D position
        kvector = np.zeros((N,N,N,ndim))
        ky, kx, kz = np.meshgrid(ks,ks,ks)
        ksquared = kx**2 + ky**2 + kz**2

    return ksquared

In [16]:
def FFT1D(x, IFT=False):
    """
    Perform the (i)FFT of input array x using the Cooley-Tukey 
    algorithm. No normalization
    """
    N = len(x)
    # change the factor when doing IFT
    if IFT:
        fi = 1
    else:
        fi = -1
        
    if N > 1:
        # split in even and odd elements
        ffteven = FFT1D(x[0::2])
        fftodd = FFT1D(x[1::2])
        
        # Exploit the period in k, vectorize instead of loop
        if N//2-1 == 0:
            k = 0 # prevent division by zero error in linspace
        else:
            k = sr.linspace(0,N//2-1,N//2)
            
        W = np.exp(fi*2j*np.pi*k/N)*fftodd
        return np.concatenate([ffteven + W
                       , ffteven-W])
    else:
        return x
        
def FFT2D(x, IFT=False):
    """
    Perform the FFT of 2D input array x 
    by nesting 1D Fourier transforms, see FFT1D(x)
    """
    FFT = np.array(np.zeros(x.shape),dtype=np.complex)
    # Take 1D fourier transform across rows
    for row in range(x.shape[0]):
        FFT[row,:] = FFT1D(x[row,:],IFT)
    # Then take 1D fourier transform across the columns
    for col in range(x.shape[1]):
        FFT[:,col] = FFT1D(FFT[:,col],IFT)
        
    return FFT

def FFT3D(x, IFT=False):
    """
    Perform the FFT of 3D input array x 
    by doing a 2D Fourier transform x.shape[0] times
    and then doing x.shape[1]*x.shape[2] 1D fourier transforms
    """
    FFT = np.array(x,dtype=np.complex)
    # Take 2D fourier transform across first axis
    for axis in range(x.shape[0]):
        FFT[axis,:,:] = FFT2D(FFT[axis,:,:],IFT)
    # Then take 1D fourier transform of the rest
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            FFT[:,i,j] = FFT1D(FFT[:,i,j],IFT)
            
    return FFT    

In [11]:
# positions = np.random.randint(0,64,(3,64**3)) # shape (3,262144)
positions = np.random.randint(0,64,(3,16**3)) # shape (3,4096)

In [12]:
kvec = kvecsquared(64,3)
kvec[0,0,0] = 1 # FT of potential at [0,0,0] is zero anyways

In [13]:
def massgrid_reduce_potential_force(positions):
    """
    Calculate the mass grid
    Reduce the values of the mesh
    Calculate the potential by going to Fourier space
    Compute the forces acting on the particles.
    
    Return the CIC object and the forces.
    """
    # Calculate the mass grid
    cic = CIC(64,positions)
    # reduce the values of the mesh
    cic.massgrid /= np.mean(cic.massgrid)
    cic.massgrid -= 1
    
    # FT of the potential is FT of the density field divided by k^2
    cic.FTdens = FFT3D(cic.massgrid)
    cic.FTdens /= kvec
    # IFFT to get the potential, don't forget normalization
    cic.potential = FFT3D(cic.FTdens,IFT=True)/cic.massgrid.size
    # It is a real value, discard imaginary part
    cic.potential = cic.potential.real
    
    # Calculate the gradient of the potential for the first 10 particles
    # using inverse CIC to interpolate the gradients of neighbouring cells
    pindices = np.array(range(0,positions.shape[1]))
    forces = cic.compute_force(pindices, cic.potential)
    
    return cic, forces

    

In [17]:
cic, forces = massgrid_reduce_potential_force(positions)

In [18]:
forces.shape

(4096, 3)

But we should do it with the dimensionless units. Too complicated for now.